Nudging Check-Ins with Geofences

Geofencing is a useful tool and concept in geographic data science and is used in many domain applications, including business, conservation, and security, as well as home automation. The concept of geofencing is very simple, yet it is a powerful technique that enriches location applications. Simply put, a geofence is a defining virtual boundary around geographic objects or an area, so that every time a user enters or leaves the boundary perimeters, actions or notifications can be triggered. With the increased use of smartphone, GPS, and location services, geofencing becomes an indispensable tool in location data analytics and intelligence. In this chapter, we will cover the following topics:

- Geofencing concept and components
- Revisiting geometry and topology (lines and polygons)
- Geofencing application example
- Geofencing and interactive web mapping

**Geofencing applications**

Geofencing applications are numerous, as we have seen in the introduction. They can be used in security, business, and smart home design, as well as a ton of other applications. However, the most widely used and common application for geofencing is marketing. In this section, we will cover the use of geofencing in marketing location-based services and products to illustrate and elaborate geofencing concepts with a real-world and concrete example. 

**Marketing and geofencing**

Advertising and marketing agencies have embraced geofencing to increase the effectiveness of their ads and reach their target users efficiently. Marketers deploy geofencing to send or display ads to users at the right time and in the right location once a user is within the perimeters of the geofence, entering it or leaving it. Although this is more likely to be in a mobile device, it also can be effectively used via desktops or tablets, triangulating from Wi-Fi locations. Let's visualize an example of two users

**Geometry and topology (lines and polygons)**

In this chapter, we will use the Brazil GPS Trajectories dataset from UCI. It contains GPS points (latitude and longitude) with timestamps. It also contains unique track_id for each trajectory

In [ ]:
# We create a GeoDataFrame using a function

def create_gdf(df, lat, lon):
  """ Convert pandas dataframe into a Geopandas GeoDataFrame"""
  crs = {'init': 'epsg:4326'}
  geometry = [Point(xy) for xy in zip(df[lon], df[lat])]
  gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
  return gdf

traj_gdf =  create_gdf(trajectories, "latitude", "longitude")
aracaju_city_points = traj_gdf[(traj_gdf['latitude']<-10.80) & (traj_gdf['longitude']>-37.5)]

In this function, we have created a trajectory GeoDataFrame, and we then took a subset of the trajectories for the city of Aracaju. This is the main file we will use for this chapter. The trajectories map can be displayed in a simple map with GeoPandas using .plot()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
aracaju_city_points.plot(markersize=5, cmap='Dark2', ax=ax)
plt.tight_layout()
plt.axis('off')
plt.savefig('aracaju_trajectory.png')
plt.show()

**Line geometries**

To create a line geometry, we need to have at least two points, and it is almost similar to how we created points in Shapely. Let's create a line from track_id_1. First, we subset the data to get only this track ID, and then we use the Shapely geometry library to create the line

In [ ]:
track_id_1 = aracaju_city_points[aracaju_city_points['track_id']== 1]

line = LineString(track_id_1.geometry)

Now that you have seen how to create one line, let's do this for the whole dataset. Remember, we said, to create line geometry, we need to filter out and take only the track IDs with two or more points before we can create a line geometry for the whole dataset

In [ ]:
filtered_trackid = aracaju_city_points.groupby('track_id').filter(lambda x: len(x) >= 2)

Then, we group all filtered_trackid instances on the track_id feature, get the geometry of the points in each track_id, and apply a Shapely LineString geometry on points in each track_id. This is a chained process, but you can take each part and reconstruct it

In [ ]:
all_tracks = filtered_trackid.groupby(['track_id'])['geometry'].apply(lambda x: LineString(x.tolist()))

In return, we get all of the lines converted out into a LineString geometry. If we want to create a GeoDataFrame out of this

In [ ]:
gdf_lines_all = gpd.GeoDataFrame(all_tracks, geometry='geometry', crs = {'init':'epsg:4326'})

gdf_lines_all['track_id'] = gdf_lines_all.index
gdf_lines_all.reset_index(drop=True, inplace=True)

Now, we have a GeoDataFrame with a LineString geometry for all track IDs. Let's have a look at the LineString map

**Polygon geometries**

To create a polygon, you need to have at least three coordinates, so it can form a triangle polygon. Let's create a polygon using Python and Shapely. In this case, I grabbed four coordinates around Aracaju city, Brazil

In [ ]:
lats = [-10.813777, -11.002150, -11.070560,-10.878416]
lons = [-37.079790, -37.203427, -37.109280, -36.986931]

To create a polygon, GeoDataFrame simply follows the same procedure we have used to create a point and line geometry. We need to first create the geometry out of the coordinates as well as the coordinate reference system

In [ ]:
crs = {'init': 'epsg:4326'}

polygon_geometry = Polygon(zip(lons, lats))

Now, we can create GeoDataFrame out of the geometry and CRS

In [ ]:
polygon_gdf = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geometry]) 

Overlayed lines and polygon GeoDataFrames


**Topology – points in a polygon**

We already covered points in a polygon with sjoin back in Chapter 3, Performing Spatial Operations Like a Pro, and will revisit it here as a topology operation. Pay attention to this example, as this is the most important aspect of the geofencing concept. To simplify our case, we will first take a buffered polygon out of one trajectory point. Later, when doing geofencing, we will bring out polygons of geographic objects, such as an airport, beach, and city center. We convert the buffered point into a polygon GeoDataFrame, reset the index, and make sure that the name of the geometry column is geometry


In [ ]:
# Create buffer on Point 20 in track_id 1
buffer = track_id_1[track_id_1['id'] == 20].buffer(0.005)
buffer.reset_index(drop=True, inplace=True)
buffer_gdf = gpd.GeoDataFrame(buffer)
buffer_gdf.columns = ['geometry']

Let's visualize the tracking points on top of the buffer polygon

In [ ]:
# Plot track_id 1 points over the Buffer Polygon 
fig, ax = plt.subplots(figsize=(10,10))
buffer_gdf.plot(ax=ax)
track_id_1.plot(ax=ax, color='black')
plt.tight_layout()
#plt.savefig('polygon_lines.png')
#plt.axis('off')
plt.show()

The following map shows the buffered polygon overlaid with track_id_1 points. As you can see, some points are inside buffer_gdf while others are outside of the polygon

Overlay track_id_1 and buffer_gdf 

As you can see, some points fall inside the buffer polygon while others are outside the polygon. We will use the topology operation (within) to check each point's position

In [ ]:
# Mask points in polygon -> returns True if inside polygon
pip_mask = track_id_1.within(buffer_gdf.loc[0, 'geometry'])
pip_mask.value_counts()

pip_mask returns a series of True or False instances to indicate whether a point is inside the polygon or not. We can count out the values of this series. In this case, 62 points fall outside the polygon, while 28 points fall inside the polygon. Let's loop over the first 12 values of pip_mask and print out whether it is inside the buffer or outside

In [ ]:
for i in pip_mask[:12]:
 if i == True:
 print('Inside the Buffer')
 else:
 print("Otuside the Buffer")

The preceding prints out a Outside the Buffer or Inside the Buffer message for each point, as shown as follows:

Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Outside the Buffer.
 Inside the Buffer.
 Inside the Buffer.
 Inside the Buffer.

Let's add these mask values to our track_id_1, variable of GeoDataFrame so that we can check it when plotting. In this example, once we filter out track_id_1 we can differentiate between the points that are inside the polygon or outside the polygon as per the following plot. Here, points have different color, depending on their position

In [ ]:
# Plot track_id 1 points over the Buffer Polygon 
fig, ax = plt.subplots(figsize=(10,10))
buffer_gdf.plot(ax=ax)
track_id_1.plot(ax=ax, column='pip_mask', cmap='bwr')
plt.tight_layout()
plt.savefig('pipmask_buffer.png')
#plt.axis('off')
plt.show()
track_id_1.loc[:,'pip_mask'] = pip_mask

**Geofencing with Plotly**

Now that we have covered all of the components of geofencing, we can perform geofencing concepts in practice using our trajectory data. We will utilize the Plotly visualization library to animate trajectories and visualize our geofencing application. We will bring three polygons that denote different use cases: an airport, beach, and city center. 

**Masking**

First, we need to create a mask where we store whether a certain point is inside the geofence or not. We first read the geofence_polygons from the dataset provided with Notebook. Upload it first in Google Colab

In [ ]:
geofence_polygons = gpd.read_file('geofence_polygons.gpkg')
geofence_polygons

Geofence polygons

Let's plot and overlay geofence_polygons and filtered_trackid points on top of each other

In [ ]:
# Plot filtered_tracid points over geofence polygons 
fig, ax = plt.subplots(figsize=(10,10))
geofence_polygons.plot(ax=ax, color='gray')
filtered_trackid.plot(ax=ax, markersize=1)
plt.tight_layout()
#plt.savefig('polygon_lines.png')
#plt.axis('off')
plt.show()

Geofence polygons (gray) and tracking points

Let's perform masking based on these three polygons in geofence_polygons. Once we do that, we filter out all points with the mask using the GeoPandas within operation

In [ ]:
mask = (geofence_polygons.loc[0, 'geometry'])| (geofence_polygons.loc[1, 'geometry']) |(geofence_polygons.loc[2, 'geometry'])


pip_mask_geofence = filtered_trackid.within(mask)
pip_mask_geofence.value_counts()

The value counts of tracking points within these three polygons are 2,257, while the other 14,420 are outside of the geofence polygons. 

Plotly interactive maps
Plotly is an analytics web application framework that provides interactive tools such as graphs and maps, and analytics and statistics tools, in the web. Recently, they have released Plotly Express, a simple interactive consistent high-level API for Python visualization. We will use Plotly Express to visualize our geofencing application. It will simply animate points, color differently when the points are inside the geofence, and give us a legend of whether a point is inside or outside of the geofence polygon

Let us import Plotly express and set up Mapbox tokens

In [ ]:
import plotly_express as px
px.set_mapbox_access_token("PASTE TOKEN HERE")

Let's first visualize a sample of tracking points with Plotly Express. We will use the scatter_mapbox function, which takes the data, latitude and longitude, and some other optional fields such as color and size

In [ ]:
px.scatter_mapbox(filtered_trackid.sample(500), 
                  lat="latitude", 
                  lon="longitude", 
                  color="geofence", 
                  size='track_id' ,
                  size_max=12, 
                  zoom=12
)

Plotly Express scatter plot for the trajectory data (color by geofence in or out)

Now that we have laid out all of the foundational work for the geofence application, the last step is to animate the points and see how the geofence works in a real-world application. scatter_mapbox has an argument parameter for that, and we only need to provide the time frame; in our case, we have a time column

In [ ]:
px.scatter_mapbox(filtered_trackid[:100],
 lat="latitude", 
 lon="longitude", 
 color="geofence", 
 size='track_id', 
 animation_frame='time',
 size_max=15, zoom=12)

You can see the final map in the Google Colab Notebook or any other Jupyter Notebook you are using. It is pretty advanced visualization with a little code that you can start experimenting with right away. Plotly Express works perfectly well with Plotly, which provides extensive tools for visualization including online and offline charts, graphs, and maps. For example, with Plotly Dash, you can build beautiful web-based analytics applications